# 1. Load data

In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import time
import operator
from operator import itemgetter

In [2]:
# Read CSV
CAB10_flow_df = pd.read_csv('~/SoftComputing/20220202_SC_CAB10_flow.csv', header=None, sep=';')
CAB10_cost_df = pd.read_csv('~/SoftComputing/20220202_SC_CAB10_cost.csv', header=None, sep=';')
CAB25_flow_df = pd.read_csv('~/SoftComputing/20220202_SC_CAB25_flow.csv', header=None, sep=';')
CAB25_cost_df = pd.read_csv('~/SoftComputing/20220202_SC_CAB25_cost.csv', header=None, sep=';')
TR55_flow_df = pd.read_csv('~/SoftComputing/20220202_SC_TR55_flow.csv', header=None, sep=';')
TR55_cost_df = pd.read_csv('~/SoftComputing/20220202_SC_TR55_cost.csv', header=None, sep=';')
TR81_flow_df = pd.read_csv('~/SoftComputing/20220202_SC_TR81_flow.csv', header=None, sep=';')
TR81_cost_df = pd.read_csv('~/SoftComputing/20220202_SC_TR81_cost.csv', header=None, sep=';')
RGP100_flow_df = pd.read_csv('~/SoftComputing/20220202_SC_RGP100_flow.csv', header=None, sep=';')
RGP100_cost_df = pd.read_csv('~/SoftComputing/20220202_SC_RGP100_cost.csv', header=None, sep=';')
RGP130_flow_df = pd.read_csv('~/SoftComputing/20220202_SC_RGP130_flow.csv', header=None, sep=';')
RGP130_cost_df = pd.read_csv('~/SoftComputing/20220202_SC_RGP130_cost.csv', header=None, sep=';')

In [3]:
# Transform csv dataframe to list
CAB10_flow = []
for i in range(0,len(CAB10_flow_df)):
    CAB10_flow.append(list(CAB10_flow_df[i]))
    
CAB10_cost = []
for i in range(0,len(CAB10_cost_df)):
    CAB10_cost.append(list(CAB10_cost_df[i]))
    
CAB25_flow = []
for i in range(0,len(CAB25_flow_df)):
    CAB25_flow.append(list(CAB25_flow_df[i]))
    
CAB25_cost = []
for i in range(0,len(CAB25_cost_df)):
    CAB25_cost.append(list(CAB25_cost_df[i]))
    
TR55_flow = []
for i in range(0,len(TR55_flow_df)):
    TR55_flow.append(list(TR55_flow_df[i]))
    
TR55_cost = []
for i in range(0,len(TR55_cost_df)):
    TR55_cost.append(list(TR55_cost_df[i]))
    
TR81_flow = []
for i in range(0,len(TR81_flow_df)):
    TR81_flow.append(list(TR81_flow_df[i]))
    
TR81_cost = []
for i in range(0,len(TR81_cost_df)):
    TR81_cost.append(list(TR81_cost_df[i]))
    
RGP100_flow = []
for i in range(0,len(RGP100_flow_df)):
    RGP100_flow.append(list(RGP100_flow_df[i]))
    
RGP100_cost = []
for i in range(0,len(RGP100_cost_df)):
    RGP100_cost.append(list(RGP100_cost_df[i]))
    
RGP130_flow = []
for i in range(0,len(RGP130_flow_df)):
    RGP130_flow.append(list(RGP130_flow_df[i]))
    
RGP130_cost = []
for i in range(0,len(RGP130_cost_df)):
    RGP130_cost.append(list(RGP130_cost_df[i]))

In [4]:
# Declare best-known solution for each case
case1_opt_sol = 491.93
case2_opt_sol = 716.98
case3_opt_sol = 312.99
case4_opt_sol = 608.14
case5_opt_sol = 767.35
case6_opt_sol = 1158.83
case7_opt_sol = 538.37
case8_opt_sol = 1034.10
case9_opt_sol = 592.64
case10_opt_sol = 853.35
case11_opt_sol = 432.19
case12_opt_sol = 778.79
case13_opt_sol = 501.85 
case14_opt_sol = 824.55
case15_opt_sol = 420.99
case16_opt_sol = 785.28
case17_opt_sol = 1231.5475
case18_opt_sol = 1418.4558
case19_opt_sol = 1179.352
case20_opt_sol = 1408.2039 
case21_opt_sol = 1252.5747
case22_opt_sol = 1433.9036
case23_opt_sol = 1211.189
case24_opt_sol = 1414.3538

# 2. Genetic Algorithm - Step-by-step code

### Step 1: Generate initial population / First generation
Generate initial population by combining random_initial_population and least_cost_initial_population

In [5]:
# Random initial population (No cost consideration)
def random_initial_solution(n_node, n_hub):
    node_list = list(range(1, n_node+1)) # Generate a list from 1 to n_nodes
    hub_list = random.sample(node_list, n_hub) # Select random n hub from node list
    solution = [0]*n_node # Generate initial list of 0 for the final solution
    
    # Place the hub
    for hub in hub_list:
        solution.pop(hub-1)
        solution.insert(hub-1, hub)
    
    # Create and shuffle spoke
    spoke_index_list = [node-1 for node in node_list if node not in hub_list]
    random.shuffle(spoke_index_list)

    for spoke_index in spoke_index_list:
        solution.pop(spoke_index)
        solution.insert(spoke_index, random.choice(hub_list))

    return solution

In [6]:
# Least-cost initial population (With cost consideration)
def least_cost_initial_solution(n_node, n_hub, cost_matrix):
    node_list = list(range(1, n_node+1)) # Generate a list from 1 to n_nodes
    hub_list = random.sample(node_list, n_hub) # Select random n hub from node list
    solution = [0]*n_node # Generate initial list of 0 for the final solution

    # Place the hub
    for hub in hub_list:
        node_list.remove(hub)
        spoke_list=node_list
    spoke_index_list = [(spoke-1) for spoke in spoke_list]
    hub_index_list = [(hub-1) for hub in hub_list]

    for hub in hub_list:
        solution.pop(hub-1)
        solution.insert(hub-1, hub)
    
    # Map each spoke to the potential hub and the respective cost between the spoke and the hub
    spoke_hub_cost={}
    for index in spoke_index_list:
        spoke_hub_cost[index+1] = [(cost_matrix[index][hub-1],hub) for hub in hub_list]
        
    # Get the hub of each spoke that requires the lowest cost
    spoke_hub_mapping_with_lowest_cost = {}
    for spoke in list(spoke_hub_cost.keys()):
        cost = [ij[0] for ij in spoke_hub_cost[spoke]]
        hub = [ij[1] for ij in spoke_hub_cost[spoke] if ij[0] == min(cost)][0]
        spoke_hub_mapping_with_lowest_cost[spoke] = hub  
        
    # Map spoke index to hub index based on selected hub
    spoke_hub_index_mapping_with_lowest_cost = [] 
    for index in spoke_index_list:
        for hub in hub_list:
            if spoke_hub_mapping_with_lowest_cost[index+1] == hub:
                spoke_hub_index_mapping_with_lowest_cost.append((index, hub-1))
    spoke_hub_index_mapping_with_lowest_cost = list(set(spoke_hub_index_mapping_with_lowest_cost))

    # Generate solution
    for index_mapping in spoke_hub_index_mapping_with_lowest_cost:
        solution.pop(index_mapping[0])
        solution.insert(index_mapping[0],index_mapping[1]+1 )

    return solution

In [7]:
def GA_initial_population_list(n_node, n_hub, n_population, initial_pop_generator_split_prob, cost_matrix):
    
    initial_population_list = []
    initial_population_generator_split = int(n_population*initial_pop_generator_split_prob)
    
    for i in range(0, initial_population_generator_split):
        solution = random_initial_solution(n_node, n_hub)
        initial_population_list.append(solution)
        
    for i in range(initial_population_generator_split, n_population):
        solution = least_cost_initial_solution(n_node, n_hub, cost_matrix)
        initial_population_list.append(solution)
        
    return initial_population_list

### Step 2: Compute the fitness of each individual in the initial population

In [8]:
def total_hub_flow(solution, node, flow_matrix):

    hub_spoke = {}
    for hub in set(solution):
        hub_spoke[hub] = [i for i,val in enumerate(solution) if val == hub] #takes the indexes of the hubs in the string
        for spokes in list(hub_spoke.values()):
            hub_spoke[hub] = [index for index in spokes if index != hub-1] #hubs and their spokes' indexes 
    hub_flows = {}
    from_hub_nodes_to_other_nodes = {} #including the parent hub and the other hubs
    for hub in set(solution):
        hub_flows[hub] = [list(flow_matrix[hub-1])] #from_hub_itself_to_other_nodes
        from_hub_nodes_to_other_nodes[hub] = [list(flow_matrix[spoke_index]) for spoke_index in hub_spoke[hub]]
        for flow_list in from_hub_nodes_to_other_nodes[hub]:
            hub_flows[hub].append(flow_list)
            
    total_hub_flow = {}
    for hub in set(solution):
        total_hub_flow[hub] = sum(list(itertools.chain.from_iterable(hub_flows[hub])))         
    return total_hub_flow

In [9]:
def total_network_cost(solution, node, flow_matrix, cost_matrix, alpha):
    total_flow_cost = 0
    for i in range(0, node):
        for j in range(0, node):
            flow_cost = flow_matrix[i][j]*(cost_matrix[i][solution[i]-1] + alpha*cost_matrix[solution[i]-1][solution[j]-1] +cost_matrix[solution[j]-1][j] )
            total_flow_cost = total_flow_cost + flow_cost
    return total_flow_cost

In [10]:
def GA_compute_fitness(population, node, flow_matrix, cost_matrix, alpha):
    solution_cost = []
    total_hub_flow_value = sum(total_hub_flow(population[0], node, flow_matrix).values())
    for solution in population:
        cost = total_network_cost(solution, node, flow_matrix, cost_matrix, alpha)/total_hub_flow_value
        solution_cost.append((cost, solution)) # Create a list of tuples containing the cost (fitness) and solution
    return solution_cost

### Step 3: Select a set of individuals to reproduce the next generation

In [11]:
# Get parents with the lowest cost
def GA_get_parents_1(GA_fitness_computation_tuple):
    min_cost_parent1 = min(chromosome[0] for chromosome in GA_fitness_computation_tuple)
    for chromosome in GA_fitness_computation_tuple:
        if chromosome[0] == min_cost_parent1:
            parent1 = chromosome[1]
        else:
            pass

    min_cost_parent2 = min(chromosome[0] for chromosome in GA_fitness_computation_tuple if chromosome[0] != min_cost_parent1)
    for chromosome in GA_fitness_computation_tuple:
        if chromosome[0] == min_cost_parent2:
            parent2 = chromosome[1]
        else:
            pass
        
    selected_parents = []
    selected_parents.append(parent1)
    selected_parents.append(parent2)
    
    return selected_parents

### Step 4: Populate offspring

#### Crossover model 1: Single-point crossover operation

In [12]:
def GA_crossover_model_1(parents, n_node):
    random_point = random.choice(range(0, n_node))
    # Declare parent1 (Select randomly)
    index_parent1 = random.randrange(0,2)
    parent1 = parents[index_parent1]
    
    if parent1 == parents[0]:
        parent2 = parents[1]
    else:
        parent2 = parents[0]
        
    offspring = [0]*n_node
    
    for i in range(0, random_point+1):
        offspring.pop(i)
        offspring.insert(i, parent1[i])

    for i in range(random_point, n_node):
        offspring.pop(i)
        offspring.insert(i, parent2[i])
        
    return offspring

### Step 5: Repair offspring to ensure all solutions are feasible

In [13]:
def GA_gene_reallocation(offspring, n_node, cost_matrix, unique_hub):
    spoke_list = list(range(1,n_node+1))
    
    # Ensure we obtain feaasible solutions only
    for hub in unique_hub:
        if offspring[hub-1] != hub:
            offspring[hub-1] = hub
        else:
            pass
    
    for hub in unique_hub:
        spoke_list.remove(hub)
    
    spoke_index_list = [(spoke-1) for spoke in spoke_list]
    hub_index_list = [(hub-1) for hub in unique_hub]

    # Map each spoke to the potential hub and the respective cost between the spoke and the hub
    spoke_hub_cost={}
    for index in spoke_index_list:
        spoke_hub_cost[index+1] = [(cost_matrix[index][hub-1],hub) for hub in unique_hub]

    else:
        pass

    # Get the hub of each spoke that requires the lowest cost
    spoke_hub_mapping_with_lowest_cost = {}
    for spoke in list(spoke_hub_cost.keys()):
        cost = [ij[0] for ij in spoke_hub_cost[spoke]]
        hub = [ij[1] for ij in spoke_hub_cost[spoke] if ij[0] == min(cost)][0]
        spoke_hub_mapping_with_lowest_cost[spoke] = hub 

    # Map spoke index to hub index based on selected hub
    spoke_hub_index_mapping_with_lowest_cost = [] 
    for index in spoke_index_list:
        for hub in unique_hub:
            if spoke_hub_mapping_with_lowest_cost[index+1] == hub:
                spoke_hub_index_mapping_with_lowest_cost.append((index, hub-1))
    spoke_hub_index_mapping_with_lowest_cost = list(set(spoke_hub_index_mapping_with_lowest_cost))

    # Generate solution
    solution = offspring.copy()
    for index_mapping in spoke_hub_index_mapping_with_lowest_cost:
        solution.pop(index_mapping[0])
        solution.insert(index_mapping[0],index_mapping[1]+1 )

    return solution

In [14]:
def GA_repair_offspring(offspring, n_node,  n_hub):
 
    # Declare input
    hubs_offspring = []
    
    # Declare unique hub in the offspring as an input for gene repair operators
    unique_hub = [hub for hub in list(set(offspring)) if hub != 0]

    # Ensure we obtain feaasible solutions only
    for hub in unique_hub:
        if offspring[hub-1] != hub:
            offspring[hub-1] = hub
        else:
            pass

    # Create a dictionary from the offspring as an input for gene repair operators
    offspring_dict = dict(zip(list(range(0,n_node)), offspring))

    # Gene repair operators
    if len(unique_hub) == n_hub: # If the number of unique hub is the same as the desired number of unique hub
        for key in offspring_dict:
            if offspring_dict[key] == 0:
                offspring[key] = random.choice(unique_hub) # Fill unassigned gene to random hub
            else:
                pass # Else, leave it as is
        offspring_repaired = GA_gene_reallocation(offspring, n_node, cost_matrix, unique_hub) # Reallocate spoke based on its lowest-cost hub
            
    elif len(unique_hub) > n_hub: # If the number of unique hub is greater than the desired number of unique hub
        repaired_hub_candidate = []
        for hub_combination in itertools.combinations(unique_hub, n_hub):
            repaired_hub_candidate.append(hub_combination) 
        repaired_unique_hub = random.choice(repaired_hub_candidate) # Reduce the number of unique hub by selecting hubs randomly
        for key in offspring_dict:
            if offspring_dict[key] == 0:
                offspring[key] = random.choice(repaired_unique_hub) # Fill unassigned gene to random hub
            elif offspring_dict[key] not in repaired_unique_hub:
                offspring[key] = random.choice(repaired_unique_hub) # Fill infeasible gene to random hub
            else:
                pass      
        offspring_repaired = GA_gene_reallocation(offspring, n_node, cost_matrix, repaired_unique_hub) # Reallocate spoke based on its lowest-cost hub
        
    elif len(unique_hub) < n_hub:
        new_hub_required = n_hub - len(unique_hub)
        possible_new_hub = list(range(1,n_node+1))
        for hub in unique_hub:
            possible_new_hub.remove(hub)
        random_new_hub = random.choices(possible_new_hub, k=new_hub_required)
        
        repaired_unique_hub = unique_hub + random_new_hub
        
        for hub in random_new_hub: # Place new hub to chromosome
            if offspring[hub-1] != hub:
                offspring[hub-1] = hub
            else:
                pass
        
        for key in offspring_dict:
            if offspring_dict[key] == 0:
                offspring[key] = random.choice(repaired_unique_hub) # Fill unassigned gene to random hub
            elif offspring_dict[key] not in repaired_unique_hub:
                offspring[key] = random.choice(repaired_unique_hub) # Fill infeasible gene to random hub
            else:
                pass  
            
        offspring_repaired = GA_gene_reallocation(offspring, n_node, cost_matrix, repaired_unique_hub) # Reallocate spoke based on its lowest-cost hub
    else:
        pass
            
    return offspring_repaired

### Step 6: Mutation

In [15]:
# Mutation operator 1: Reassign random spoke to one of the available hubs
def GA_mutation_operator_1(offspring, n_node, mutation_rate):
    unique_hub = list(set(offspring))
    node_list = list(range(1,n_node+1))
    
    for hub in unique_hub:
        spoke_index_list = [node-1 for node in node_list if node not in unique_hub]
    
    mutation_counter = mutation_rate * n_node
    counter = 0
    mutated_offspring = offspring.copy()
    while counter < mutation_counter:
        node_index_to_reassign = random.choice(spoke_index_list)
        mutated_offspring[node_index_to_reassign] = random.choice([hub for hub in unique_hub if hub != offspring[node_index_to_reassign]])
        counter += 1
    
    return mutated_offspring

In [16]:
# Mutation operator 2: Swap random spoke
def GA_mutation_operator_2(offspring, n_node, mutation_rate):
    unique_hub = list(set(offspring))
    node_list = list(range(1,n_node+1))
    
    for hub in unique_hub:
        spoke_index_list = [node-1 for node in node_list if node not in unique_hub]

    mutation_counter = mutation_rate * n_node
    counter = 0
    mutated_offspring = offspring.copy()
    while counter < mutation_counter:
        node_index_to_swap = random.choices(spoke_index_list,k=2)
        mutated_offspring[node_index_to_swap[0]] = offspring[node_index_to_swap[1]]
        mutated_offspring[node_index_to_swap[1]] = offspring[node_index_to_swap[0]]
        counter += 1
    
    return mutated_offspring

In [20]:
def GA_mutation(offspring, n_node, mutation_rate):
    chosen_operation = random.choice(list(range(1,3)))
    if chosen_operation == 1:
        return GA_mutation_operator_1(offspring, n_node, mutation_rate)
    elif chosen_operation == 2:
        return GA_mutation_operator_2(offspring, n_node, mutation_rate)
    else:
        pass
    return mutated_offspring

### Step 7: Genetic algorithm full code

In [17]:
def genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter):
    
    # Step 1: Declare inputs
    generation_count = 0
    optimal_solution_list = []
    
    # Step 2: Generate initial population
    initial_population_df = pd.DataFrame()
    counter = 2
    initial_population_counter = 0
    while initial_population_counter < counter:
        initial_population_temp = GA_initial_population_list(n_node, n_hub, n_population, initial_pop_generator_split_prob, cost_matrix)
        total_hub_flow_value = sum(total_hub_flow(initial_population_temp[0], n_node, flow_matrix).values())

        initial_population_temp_fitness = GA_compute_fitness(initial_population_temp, n_node, flow_matrix, cost_matrix, alpha)

        initial_population_temp_df = pd.DataFrame(initial_population_temp_fitness, columns=['fitness','config'])
        initial_population_counter += 1

    initial_population_df = initial_population_temp_df.sort_values('fitness',ascending=True).head(n_population)
    initial_population = initial_population_df['config'].to_list()
    
    # Step 3: Compute the fitness of each chromosome in initial population
    initial_population_fitness = [tuple(x) for x in initial_population_df.to_numpy()]
    
    old_gen = initial_population
    old_gen_fitness = initial_population_fitness
    print('Initial population is created')
    print('Initial population evaluation is done')
    
    while generation_count < total_generation:
        # Step 3: Select a set of individuals as parents (Tournamet selection)
        parents = GA_get_parents_1(old_gen_fitness) # Least cost parents
        
        crossover_random_prob = random.uniform(0, 1)
        if crossover_random_prob <= crossover_prob:
            # Step 4: Populate offspring
            counter = 0
            while counter < crossover_counter:
                offspring_crossovered = GA_crossover_model_1(parents, n_node)
                offspring_repaired = GA_repair_offspring(offspring_crossovered, n_node, n_hub)
                offspring_repaired_fitness = total_network_cost(offspring_repaired, n_node, flow_matrix, cost_matrix, alpha)/total_hub_flow_value
                print(f'Offspring population {generation_count+1} is done')
                print(f'Offspring repaired fitness: {round(offspring_repaired_fitness,2)}')

                # Step 5: Offspring mutation
                offspring_mutation = GA_mutation(offspring_repaired, n_node, mutation_rate)
                offspring_mutation_fitness = total_network_cost(offspring_mutation, n_node, flow_matrix, cost_matrix, alpha)/total_hub_flow_value
                print(f'Offspring mutation fitness: {round(offspring_mutation_fitness,2)}')

                # Step 6: Replace the worst fit chromosome in the last generation with the new offspring
                old_gen_worst_fitness = max(old_gen_fitness, key=itemgetter(0))[0]
                old_gen_worst_fitness_chromosome = max(old_gen_fitness, key=itemgetter(0))[1]
                print(f'Previous generation worst fitness: {round(old_gen_worst_fitness,2)}')

                # If offspring_repaired is better than old_gen and offspring_mutation, replace the worst chromosome in old_gen with offspring_repaired
                if offspring_repaired_fitness < old_gen_worst_fitness and offspring_repaired_fitness <= offspring_mutation_fitness:
                    new_gen = [offspring_repaired if old_chromosome == old_gen_worst_fitness_chromosome else old_chromosome for old_chromosome in old_gen]
                # If offspring_mutation is better than old_gen and offspring_repaired, replace the worst chromosome in old_gen with offspring_mutation
                elif offspring_mutation_fitness < old_gen_worst_fitness and offspring_mutation_fitness <= offspring_repaired_fitness:
                    new_gen = [offspring_mutation if old_chromosome == old_gen_worst_fitness_chromosome else old_chromosome for old_chromosome in old_gen]
                # Else, keep the old_gen
                else:
                    new_gen = old_gen
                    
                old_gen = new_gen
                old_gen_fitness = GA_compute_fitness(new_gen, n_node, flow_matrix, cost_matrix, alpha)
                old_gen_worst_fitness = max(old_gen_fitness, key=itemgetter(0))[0]
                print(f'Replacement {counter+1} of iteration {generation_count+1} is done')
                counter += 1
                
            # Step 7: Calculate optimal solution
            new_gen_fitness = GA_compute_fitness(new_gen, n_node, flow_matrix, cost_matrix, alpha)

            for index in list(range(0,n_population)):
                optimal_solution_cost = new_gen_fitness[index][0]
                optimal_solution_node = new_gen[index]
                optimal_solution_hub = list(set(new_gen[index]))
                optimal_solution_list.append((optimal_solution_cost, optimal_solution_node, optimal_solution_hub))

            print(f'Optimal solution of iteration {generation_count+1} has been calculated')
            print('-----------------------------------------------------------------------')
        else:
            # Step 4: Select one of the parents randomly
            random_parent = parents[0]
            random_parent_fitness = total_network_cost(random_parent, n_node, flow_matrix, cost_matrix, alpha)/total_hub_flow_value
            
            # Step 5: Perform mutation if applicable
            mutation_random_prob = random.uniform(0,1)
            if mutation_random_prob <= mutation_prob:
                offspring_mutation = GA_mutation(random_parent, n_node, mutation_rate)
                offspring_mutation_fitness = total_network_cost(offspring_mutation, n_node, flow_matrix, cost_matrix, alpha)/total_hub_flow_value
                print(f'Offspring mutation fitness: {round(offspring_mutation_fitness,2)}')
                
                # Step 6: Replace the worst fit chromosome in the last generation with the new offspring
                old_gen_worst_fitness = max(old_gen_fitness, key=itemgetter(0))[0]
                old_gen_worst_fitness_chromosome = max(old_gen_fitness, key=itemgetter(0))[1]
                print(f'Previous generation worst fitness: {round(old_gen_worst_fitness,2)}')
                
                if offspring_mutation_fitness < old_gen_worst_fitness and offspring_mutation_fitness <= random_parent_fitness:
                    new_gen = [offspring_mutation if old_chromosome == old_gen_worst_fitness_chromosome else old_chromosome for old_chromosome in old_gen]
                elif random_parent_fitness < old_gen_worst_fitness and random_parent_fitness <= offspring_mutation_fitness:
                    new_gen = [random_parent if old_chromosome == old_gen_worst_fitness_chromosome else old_chromosome for old_chromosome in old_gen]
                else:
                    new_gen = old_gen    
                
                # Step 7: Calculate optimal solution
                new_gen_fitness = GA_compute_fitness(new_gen, n_node, flow_matrix, cost_matrix, alpha)

                for index in list(range(0,n_population)):
                    optimal_solution_cost = new_gen_fitness[index][0]
                    optimal_solution_node = new_gen[index]
                    optimal_solution_hub = list(set(new_gen[index]))
                    optimal_solution_list.append((optimal_solution_cost, optimal_solution_node, optimal_solution_hub))

                print(f'Optimal solution of iteration {generation_count+1} has been calculated')
                print('-----------------------------------------------------------------------')
                
            else:
                offspring_random = random_parent
                offspring_random_fitness = total_network_cost(offspring_random, n_node, flow_matrix, cost_matrix, alpha)/total_hub_flow_value
                print(f'Random parent fitness: {round(offspring_random_fitness,2)}')
                
                # Step 6: Replace the worst fit chromosome in the last generation with the new offspring
                old_gen_worst_fitness = max(old_gen_fitness, key=itemgetter(0))[0]
                old_gen_worst_fitness_chromosome = max(old_gen_fitness, key=itemgetter(0))[1]
                print(f'Previous generation worst fitness: {round(old_gen_worst_fitness,2)}')
                
                new_gen = [offspring_random if old_chromosome == old_gen_worst_fitness_chromosome else old_chromosome for old_chromosome in old_gen]
        
                # Step 7: Calculate optimal solution
                new_gen_fitness = GA_compute_fitness(new_gen, n_node, flow_matrix, cost_matrix, alpha)

                for index in list(range(0,n_population)):
                    optimal_solution_cost = new_gen_fitness[index][0]
                    optimal_solution_node = new_gen[index]
                    optimal_solution_hub = list(set(new_gen[index]))
                    optimal_solution_list.append((optimal_solution_cost, optimal_solution_node, optimal_solution_hub))

                print(f'Optimal solution of iteration {generation_count+1} has been calculated')
                print('-----------------------------------------------------------------------')
        
        # Step 8: Looping/Regeneration
        old_gen = new_gen
        old_gen_fitness = GA_compute_fitness(new_gen, n_node, flow_matrix, cost_matrix, alpha)
        old_gen_worst_fitness = max(old_gen_fitness, key=itemgetter(0))[0]
        generation_count += 1
            
    global_best = round(min(optimal_solution_list, key=itemgetter(0))[0],2)
    best_config = min(optimal_solution_list, key=itemgetter(0))[1]
    selected_hubs = min(optimal_solution_list, key=itemgetter(0))[2]
    
    global_best_solution = [global_best, best_config, selected_hubs]

    print(f'Global best value: {global_best}')
    return global_best_solution

### Step 8: Genetic Algorithm Implementation

#### CASE 5: CAB dataset, n_node = 25, n_hub = 3, alpha = 0.2

In [220]:
%%time

start_time = time.time()

n_node = 25
n_hub = 3
n_population = 25
initial_pop_generator_split_prob = 0
flow_matrix = CAB25_flow
cost_matrix = CAB25_cost
alpha = 0.2
total_generation = 10
crossover_prob = 0.9
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case5_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case5_df["cost"].mean(),2)
best_tnc = round(case5_df["cost"].min(),2)
gap = round((best_tnc - case5_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case5_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 835.09
Offspring mutation fitness: 1092.29
Previous generation worst fitness: 1615.93
Replacement 1 of iteration 1 is done
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring population 2 is done
Offspring repaired fitness: 1229.37
Offspring mutation fitness: 1491.35
Previous generation worst fitness: 1593.99
Replacement 1 of iteration 2 is done
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring population 3 is done
Offspring repaired fitness: 866.4
Offspring mutation fitness: 1067.19
Previous generation worst fitness: 1538.46
Replacement 1 of iteration 3 is done
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Offspring popu

#### CASE 6: CAB dataset, n_node = 25, n_hub = 3, alpha = 0.8

In [224]:
%%time

start_time = time.time()

n_node = 25
n_hub = 3
n_population = 25
initial_pop_generator_split_prob = 0.5
flow_matrix = CAB25_flow
cost_matrix = CAB25_cost
alpha = 0.8
total_generation = 5
crossover_prob = 0.9
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case6_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case6_df["cost"].mean(),2)
best_tnc = round(case6_df["cost"].min(),2)
gap = round((best_tnc - case6_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case6_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 1194.27
Offspring mutation fitness: 1280.03
Previous generation worst fitness: 3730.64
Replacement 1 of iteration 1 is done
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring population 2 is done
Offspring repaired fitness: 1289.46
Offspring mutation fitness: 1489.07
Previous generation worst fitness: 3414.63
Replacement 1 of iteration 2 is done
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring population 3 is done
Offspring repaired fitness: 1400.68
Offspring mutation fitness: 1431.85
Previous generation worst fitness: 3146.16
Replacement 1 of iteration 3 is done
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Random pare

#### CASE 7: CAB dataset, n_node = 25, n_hub = 5, alpha = 0.2 (Descending)

In [85]:
%%time

start_time = time.time()

n_node = 25
n_hub = 5
n_population = 1000
initial_pop_generator_split_prob = 0.1
flow_matrix = CAB25_flow
cost_matrix = CAB25_cost
alpha = 0.2
total_generation = 5
crossover_prob = 0.9
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 10

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case7_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case7_df["cost"].mean(),2)
best_tnc = round(case7_df["cost"].min(),2)
gap = round((best_tnc - case7_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case7_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 641.75
Offspring mutation fitness: 846.43
Previous generation worst fitness: 3133.23
Replacement 1 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 829.28
Offspring mutation fitness: 885.0
Previous generation worst fitness: 2818.96
Replacement 2 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 633.48
Offspring mutation fitness: 900.89
Previous generation worst fitness: 2754.94
Replacement 3 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 658.48
Offspring mutation fitness: 733.94
Previous generation worst fitness: 2720.49
Replacement 4 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 694.28
Offspring mutation fitness: 945.78
Previous generation worst fitness: 2715.85
Replacement 5 of iteration 1 is done
Offspring population 1 is do

#### CASE 8: CAB dataset, n_node = 25, n_hub = 5, alpha = 0.8

In [51]:
%%time

start_time = time.time()

n_node = 25
n_hub = 5
n_population = 1000
initial_pop_generator_split_prob = 0.1
flow_matrix = CAB25_flow
cost_matrix = CAB25_cost
alpha = 0.8
total_generation = 25
crossover_prob = 0.95
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case8_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case8_df["cost"].mean(),2)
best_tnc = round(case8_df["cost"].min(),2)
gap = round((best_tnc - case8_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case8_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 1104.01
Offspring mutation fitness: 1355.88
Previous generation worst fitness: 3625.7
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring population 2 is done
Offspring repaired fitness: 1104.01
Offspring mutation fitness: 1465.23
Previous generation worst fitness: 3590.42
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring population 3 is done
Offspring repaired fitness: 1189.17
Offspring mutation fitness: 1409.86
Previous generation worst fitness: 3514.39
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Offspring population 4 is done
Offspring repaired fitness: 1106.73
Offspring mutation fitness: 1287.17
Previous generation 

#### CASE 9: TR dataset, n_node = 55, n_hub = 3, alpha = 0.2

In [55]:
%%time

start_time = time.time()

n_node = 55
n_hub = 3
n_population = 1000
initial_pop_generator_split_prob = 0.1
flow_matrix = TR55_flow
cost_matrix = TR55_cost
alpha = 0.2
total_generation = 10
crossover_prob = 0.95
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case9_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case9_df["cost"].mean(),2)
best_tnc = round(case9_df["cost"].min(),2)
gap = round((best_tnc - case9_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case9_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 623.68
Offspring mutation fitness: 721.08
Previous generation worst fitness: 2290.14
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring population 2 is done
Offspring repaired fitness: 749.78
Offspring mutation fitness: 769.59
Previous generation worst fitness: 2152.2
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring population 3 is done
Offspring repaired fitness: 623.68
Offspring mutation fitness: 666.57
Previous generation worst fitness: 2113.86
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Offspring population 4 is done
Offspring repaired fitness: 635.96
Offspring mutation fitness: 673.52
Previous generation worst fi

#### CASE 10: TR dataset, n_node = 55, n_hub = 3, alpha = 0.8

In [57]:
%%time

start_time = time.time()

n_node = 55 # Change
n_hub = 3 # Change
n_population = 1000 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = TR55_flow # Change
cost_matrix = TR55_cost # Change
alpha = 0.8 # Change
total_generation = 5
crossover_prob = 0.95
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case10_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case10_df["cost"].mean(),2)
best_tnc = round(case10_df["cost"].min(),2)
gap = round((best_tnc - case10_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case10_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 914.79
Offspring mutation fitness: 1065.89
Previous generation worst fitness: 2704.48
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring population 2 is done
Offspring repaired fitness: 914.79
Offspring mutation fitness: 963.49
Previous generation worst fitness: 2642.67
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring population 3 is done
Offspring repaired fitness: 1008.45
Offspring mutation fitness: 1064.91
Previous generation worst fitness: 2621.65
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Offspring population 4 is done
Offspring repaired fitness: 914.79
Offspring mutation fitness: 1043.42
Previous generation wor

#### CASE 11: TR dataset, n_node = 55, n_hub = 5, alpha = 0.2

In [186]:
%%time

start_time = time.time()

n_node = 55 # Change
n_hub = 5 # Change
n_population = 3000 # Change
initial_pop_generator_split_prob = 0
flow_matrix = TR55_flow # Change
cost_matrix = TR55_cost # Change
alpha = 0.2 # Change
total_generation = 1
crossover_prob = 0.01
mutation_prob = 0.01
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case11_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case11_df["cost"].mean(),2)
best_tnc = round(case11_df["cost"].min(),2)
gap = round((best_tnc - case11_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case11_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Random parent fitness: 470.79
Previous generation worst fitness: 1720.5
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Global best value: 470.79
====================== REPLICATION 0 IS FINISHED ======================



Initial population is created
Initial population evaluation is done
Random parent fitness: 492.99
Previous generation worst fitness: 1934.91
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Global best value: 492.99
====================== REPLICATION 1 IS FINISHED ======================



Initial population is created
Initial population evaluation is done
Random parent fitness: 458.14
Previous generation worst fitness: 1866.17
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Glob

#### CASE 12: TR dataset, n_node = 55, n_hub = 5, alpha = 0.8

In [75]:
%%time

start_time = time.time()

n_node = 55 # Change
n_hub = 5 # Change
n_population = 400 # Change
initial_pop_generator_split_prob = 0
flow_matrix = TR55_flow # Change
cost_matrix = TR55_cost # Change
alpha = 0.8 # Change
total_generation = 10
crossover_prob = 0.8
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case12_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case12_df["cost"].mean(),2)
best_tnc = round(case12_df["cost"].min(),2)
gap = round((best_tnc - case12_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case12_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 854.73
Offspring mutation fitness: 910.93
Previous generation worst fitness: 1748.48
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Previous generation worst fitness: 1672.69
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring population 3 is done
Offspring repaired fitness: 831.65
Offspring mutation fitness: 895.26
Previous generation worst fitness: 1636.31
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Offspring population 4 is done
Offspring repaired fitness: 841.11
Offspring mutation fitness: 981.24
Previous generation worst fitness: 1625.04
Optimal solution of iteration 4 has been calculated
---------------------------------

#### CASE 13: TR dataset, n_node = 81, n_hub = 5, alpha = 0.2

In [28]:
%%time

start_time = time.time()

n_node = 81 # Change
n_hub = 5 # Change
n_population = 200 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = TR81_flow # Change
cost_matrix = TR81_cost # Change
alpha = 0.2 # Change
total_generation = 1
crossover_prob = 0.8
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 10

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case13_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case13_df["cost"].mean(),2)
best_tnc = round(case13_df["cost"].min(),2)
gap = round((best_tnc - case13_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case13_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 674.32
Offspring mutation fitness: 769.1
Previous generation worst fitness: 1978.79
Replacement 1 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 655.76
Offspring mutation fitness: 841.21
Previous generation worst fitness: 1853.93
Replacement 2 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 637.74
Offspring mutation fitness: 773.44
Previous generation worst fitness: 1659.47
Replacement 3 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 796.66
Offspring mutation fitness: 856.94
Previous generation worst fitness: 1653.37
Replacement 4 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 717.75
Offspring mutation fitness: 780.57
Previous generation worst fitness: 1650.02
Replacement 5 of iteration 1 is done
Offspring population 1 is do

#### CASE 14: TR dataset, n_node = 81, n_hub = 5, alpha = 0.8

In [32]:
%%time

start_time = time.time()

n_node = 81 # Change
n_hub = 5 # Change
n_population = 400 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = TR81_flow # Change
cost_matrix = TR81_cost # Change
alpha = 0.8 # Change
total_generation = 1
crossover_prob = 0.8
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 10

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case14_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case14_df["cost"].mean(),2)
best_tnc = round(case14_df["cost"].min(),2)
gap = round((best_tnc - case14_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case14_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Random parent fitness: 908.44
Previous generation worst fitness: 2578.6
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Global best value: 908.44
====================== REPLICATION 0 IS FINISHED ======================



Initial population is created
Initial population evaluation is done
Random parent fitness: 908.04
Previous generation worst fitness: 2617.91
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Global best value: 908.04
====================== REPLICATION 1 IS FINISHED ======================



Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 889.66
Offspring mutation fitness: 1202.54
Previous generation worst fitness: 2478.6
Replacement 1 of iteration 1 is done
Offspring population

#### CASE 15: TR dataset, n_node = 81, n_hub = 7, alpha = 0.2

In [35]:
%%time

start_time = time.time()

n_node = 81 # Change
n_hub = 7 # Change
n_population = 400 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = TR81_flow # Change
cost_matrix = TR81_cost # Change
alpha = 0.2 # Change
total_generation = 1
crossover_prob = 0.8
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 10

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case15_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case15_df["cost"].mean(),2)
best_tnc = round(case15_df["cost"].min(),2)
gap = round((best_tnc - case15_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case15_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 593.48
Offspring mutation fitness: 708.01
Previous generation worst fitness: 1961.07
Replacement 1 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 506.14
Offspring mutation fitness: 582.39
Previous generation worst fitness: 1923.8
Replacement 2 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 509.23
Offspring mutation fitness: 608.38
Previous generation worst fitness: 1811.36
Replacement 3 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 564.73
Offspring mutation fitness: 691.1
Previous generation worst fitness: 1802.11
Replacement 4 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 426.31
Offspring mutation fitness: 519.2
Previous generation worst fitness: 1787.36
Replacement 5 of iteration 1 is done
Offspring population 1 is done

#### CASE 16: TR dataset, n_node = 81, n_hub = 7, alpha = 0.8

In [39]:
%%time

start_time = time.time()

n_node = 81 # Change
n_hub = 7 # Change
n_population = 400 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = TR81_flow # Change
cost_matrix = TR81_cost # Change
alpha = 0.8 # Change
total_generation = 1
crossover_prob = 0.8
mutation_prob = 0.8
mutation_rate = 0.1
crossover_counter = 10

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case16_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case16_df["cost"].mean(),2)
best_tnc = round(case16_df["cost"].min(),2)
gap = round((best_tnc - case16_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case16_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 858.4
Offspring mutation fitness: 972.26
Previous generation worst fitness: 2551.63
Replacement 1 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 937.18
Offspring mutation fitness: 996.41
Previous generation worst fitness: 2388.8
Replacement 2 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 903.18
Offspring mutation fitness: 998.25
Previous generation worst fitness: 2359.17
Replacement 3 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 874.89
Offspring mutation fitness: 998.16
Previous generation worst fitness: 2346.46
Replacement 4 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 915.19
Offspring mutation fitness: 1159.96
Previous generation worst fitness: 2321.34
Replacement 5 of iteration 1 is done
Offspring population 1 is do

#### CASE 17: RGP dataset, n_node = 100, n_hub = 7, alpha = 0.2

In [41]:
%%time

start_time = time.time()

n_node = 100 # Change
n_hub = 7 # Change
n_population = 400 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = RGP100_flow # Change
cost_matrix = RGP100_cost # Change
alpha = 0.2 # Change
total_generation = 1
crossover_prob = 0.8
mutation_prob = 0.8
mutation_rate = 0.1
crossover_counter = 10

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case17_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case17_df["cost"].mean(),2)
best_tnc = round(case17_df["cost"].min(),2)
gap = round((best_tnc - case17_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case17_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 1318.0
Offspring mutation fitness: 1338.25
Previous generation worst fitness: 1562.63
Replacement 1 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1301.05
Offspring mutation fitness: 1335.66
Previous generation worst fitness: 1561.55
Replacement 2 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1297.81
Offspring mutation fitness: 1339.55
Previous generation worst fitness: 1553.89
Replacement 3 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1338.62
Offspring mutation fitness: 1347.95
Previous generation worst fitness: 1551.28
Replacement 4 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1328.06
Offspring mutation fitness: 1361.17
Previous generation worst fitness: 1547.5
Replacement 5 of iteration 1 is done
Offspring populatio

#### CASE 18: RGP dataset, n_node = 100, n_hub = 7, alpha = 0.8

In [43]:
%%time

start_time = time.time()

n_node = 100 # Change
n_hub = 7 # Change
n_population = 400 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = RGP100_flow # Change
cost_matrix = RGP100_cost # Change
alpha = 0.8 # Change
total_generation = 1
crossover_prob = 0.8
mutation_prob = 0.8
mutation_rate = 0.1
crossover_counter = 10

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case18_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case18_df["cost"].mean(),2)
best_tnc = round(case18_df["cost"].min(),2)
gap = round((best_tnc - case18_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case18_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 1679.43
Offspring mutation fitness: 1706.13
Previous generation worst fitness: 1982.5
Replacement 1 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1689.53
Offspring mutation fitness: 1714.67
Previous generation worst fitness: 1959.18
Replacement 2 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1680.28
Offspring mutation fitness: 1722.4
Previous generation worst fitness: 1951.67
Replacement 3 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1699.43
Offspring mutation fitness: 1722.95
Previous generation worst fitness: 1950.44
Replacement 4 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1702.32
Offspring mutation fitness: 1744.07
Previous generation worst fitness: 1943.15
Replacement 5 of iteration 1 is done
Offspring populatio

#### CASE 19: RGP dataset, n_node = 100, n_hub = 10, alpha = 0.2

In [73]:
%%time

start_time = time.time()

n_node = 100 # Change
n_hub = 10 # Change
n_population = 400 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = RGP100_flow # Change
cost_matrix = RGP100_cost # Change
alpha = 0.2 # Change
total_generation = 1
crossover_prob = 0.8
mutation_prob = 0.8
mutation_rate = 0.1
crossover_counter = 10

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case19_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case19_df["cost"].mean(),2)
best_tnc = round(case19_df["cost"].min(),2)
gap = round((best_tnc - case19_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case19_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 1259.92
Offspring mutation fitness: 1298.15
Previous generation worst fitness: 1515.16
Replacement 1 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1291.73
Offspring mutation fitness: 1333.04
Previous generation worst fitness: 1512.3
Replacement 2 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1284.16
Offspring mutation fitness: 1322.83
Previous generation worst fitness: 1510.27
Replacement 3 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1258.8
Offspring mutation fitness: 1318.47
Previous generation worst fitness: 1506.99
Replacement 4 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1266.7
Offspring mutation fitness: 1304.48
Previous generation worst fitness: 1506.08
Replacement 5 of iteration 1 is done
Offspring population

#### CASE 20: RGP dataset, n_node = 100, n_hub = 10, alpha = 0.8

In [47]:
%%time

start_time = time.time()

n_node = 100 # Change
n_hub = 10 # Change
n_population = 400 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = RGP100_flow # Change
cost_matrix = RGP100_cost # Change
alpha = 0.8 # Change
total_generation = 1
crossover_prob = 0.8
mutation_prob = 0.8
mutation_rate = 0.1
crossover_counter = 10

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case20_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case20_df["cost"].mean(),2)
best_tnc = round(case20_df["cost"].min(),2)
gap = round((best_tnc - case20_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case20_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 1676.59
Offspring mutation fitness: 1711.19
Previous generation worst fitness: 1950.14
Replacement 1 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1667.45
Offspring mutation fitness: 1689.35
Previous generation worst fitness: 1930.55
Replacement 2 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1643.53
Offspring mutation fitness: 1677.19
Previous generation worst fitness: 1926.89
Replacement 3 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1675.62
Offspring mutation fitness: 1695.6
Previous generation worst fitness: 1916.47
Replacement 4 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1643.98
Offspring mutation fitness: 1688.54
Previous generation worst fitness: 1909.54
Replacement 5 of iteration 1 is done
Offspring populati

#### CASE 21: RGP dataset, n_node = 130, n_hub = 7, alpha = 0.2

In [50]:
%%time

start_time = time.time()

n_node = 130 # Change
n_hub =7 # Change
n_population = 400 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = RGP130_flow # Change
cost_matrix = RGP130_cost # Change
alpha = 0.2 # Change
total_generation = 1
crossover_prob = 0.8
mutation_prob = 0.8
mutation_rate = 0.1
crossover_counter = 20

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case21_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case21_df["cost"].mean(),2)
best_tnc = round(case21_df["cost"].min(),2)
gap = round((best_tnc - case21_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case21_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 1357.07
Offspring mutation fitness: 1381.78
Previous generation worst fitness: 1578.61
Replacement 1 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1343.95
Offspring mutation fitness: 1362.63
Previous generation worst fitness: 1578.06
Replacement 2 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1362.51
Offspring mutation fitness: 1384.85
Previous generation worst fitness: 1570.33
Replacement 3 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1349.89
Offspring mutation fitness: 1380.88
Previous generation worst fitness: 1565.21
Replacement 4 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1338.86
Offspring mutation fitness: 1362.33
Previous generation worst fitness: 1564.79
Replacement 5 of iteration 1 is done
Offspring populat

#### CASE 22: RGP dataset, n_node = 130, n_hub = 7, alpha = 0.8

In [55]:
%%time

start_time = time.time()

n_node = 130 # Change
n_hub =7 # Change
n_population = 500 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = RGP130_flow # Change
cost_matrix = RGP130_cost # Change
alpha = 0.8 # Change
total_generation = 1
crossover_prob = 0.8
mutation_prob = 0.8
mutation_rate = 0.1
crossover_counter = 20

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case22_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case22_df["cost"].mean(),2)
best_tnc = round(case22_df["cost"].min(),2)
gap = round((best_tnc - case22_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case22_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 1704.6
Offspring mutation fitness: 1736.97
Previous generation worst fitness: 1987.67
Replacement 1 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1722.93
Offspring mutation fitness: 1752.31
Previous generation worst fitness: 1978.07
Replacement 2 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1720.15
Offspring mutation fitness: 1762.03
Previous generation worst fitness: 1971.23
Replacement 3 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1731.16
Offspring mutation fitness: 1761.22
Previous generation worst fitness: 1970.68
Replacement 4 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1709.91
Offspring mutation fitness: 1721.5
Previous generation worst fitness: 1965.08
Replacement 5 of iteration 1 is done
Offspring populatio

#### CASE 23: RGP dataset, n_node = 130, n_hub = 10, alpha = 0.2

In [57]:
%%time

start_time = time.time()

n_node = 130 # Change
n_hub = 10 # Change
n_population = 500 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = RGP130_flow # Change
cost_matrix = RGP130_cost # Change
alpha = 0.2 # Change
total_generation = 1
crossover_prob = 0.8
mutation_prob = 0.8
mutation_rate = 0.1
crossover_counter = 20

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case23_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case23_df["cost"].mean(),2)
best_tnc = round(case23_df["cost"].min(),2)
gap = round((best_tnc - case23_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case23_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 1305.14
Offspring mutation fitness: 1326.03
Previous generation worst fitness: 1559.33
Replacement 1 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1318.93
Offspring mutation fitness: 1342.85
Previous generation worst fitness: 1544.58
Replacement 2 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1327.92
Offspring mutation fitness: 1385.56
Previous generation worst fitness: 1543.12
Replacement 3 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1306.51
Offspring mutation fitness: 1344.5
Previous generation worst fitness: 1539.8
Replacement 4 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1300.64
Offspring mutation fitness: 1350.57
Previous generation worst fitness: 1538.84
Replacement 5 of iteration 1 is done
Offspring populatio

#### CASE 24: RGP dataset, n_node = 130, n_hub = 10, alpha = 0.8

In [59]:
%%time

start_time = time.time()

n_node = 130 # Change
n_hub = 10 # Change
n_population = 500 # Change
initial_pop_generator_split_prob = 0.1
flow_matrix = RGP130_flow # Change
cost_matrix = RGP130_cost # Change
alpha = 0.8 # Change
total_generation = 1
crossover_prob = 0.8
mutation_prob = 0.8
mutation_rate = 0.1
crossover_counter = 20

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, alpha, total_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case24_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case24_df["cost"].mean(),2)
best_tnc = round(case24_df["cost"].min(),2)
gap = round((best_tnc - case24_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case24_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring population 1 is done
Offspring repaired fitness: 1670.77
Offspring mutation fitness: 1706.98
Previous generation worst fitness: 1966.66
Replacement 1 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1719.93
Offspring mutation fitness: 1749.11
Previous generation worst fitness: 1963.62
Replacement 2 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1683.84
Offspring mutation fitness: 1721.43
Previous generation worst fitness: 1961.4
Replacement 3 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1716.76
Offspring mutation fitness: 1766.59
Previous generation worst fitness: 1959.1
Replacement 4 of iteration 1 is done
Offspring population 1 is done
Offspring repaired fitness: 1686.71
Offspring mutation fitness: 1750.53
Previous generation worst fitness: 1959.05
Replacement 5 of iteration 1 is done
Offspring populatio